## Data Preprocessing

### Importing the dataset

In [3]:
import pandas as pd


# Step 1: Read the CSV file with appropriate parameters
df = pd.read_csv('../data/movie_dataset.csv')

df = df.head(50)
# print(dataset)

# summary_statictics = dataset.describe(include='all')

df
# print(summary_statictics)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
5,5,258000000,Fantasy Action Adventure,http://www.sonypictures.com/movies/spider-man3/,559,dual identity amnesia sandstorm love of one's ...,en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,...,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576,Tobey Maguire Kirsten Dunst James Franco Thoma...,"[{'name': 'Francine Maisler', 'gender': 1, 'de...",Sam Raimi
6,6,260000000,Animation Family,http://disney.go.com/disneypictures/tangled/,38757,hostage magic horse fairy tale musical,en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,...,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330,Zachary Levi Mandy Moore Donna Murphy Ron Perl...,"[{'name': 'John Lasseter', 'gender': 2, 'depar...",Byron Howard
7,7,280000000,Action Adventure Science Fiction,http://marvel.com/movies/movie/193/avengers_ag...,99861,marvel comic sequel superhero based on comic b...,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,134.279229,...,141.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Age Has Come.,Avengers: Age of Ultron,7.3,6767,Robert Downey Jr. Chris Hemsworth Mark Ruffalo...,"[{'name': 'Danny Elfman', 'gender': 2, 'depart...",Joss Whedon
8,8,250000000,Adventure Fantasy Family,http://harrypotter.warnerbros.com/harrypottera...,767,witch magic broom

## Visualising data

In [4]:
# import matplotlib.pyplot as plt

# df['release_date'] = pd.to_datetime(df['release_date'], format='%Y/%m/%d')
# df['release_year'] = df['release_date'].dt.year

# plt.figure(figsize=(12, 8))
# plt.scatter(df['release_year'], df['vote_count'], color='blue', alpha=0.5)
# plt.xlabel('Release Year')
# plt.ylabel('Vote Average')
# plt.title('Movie Budget vs Vote Average')
# # plt.xscale('log')  # Optional: Use logarithmic scale for better visualization if there are large ranges
# # plt.yscale('log')  # Optional: Use logarithmic scale for better visualization if there are large ranges
# plt.tight_layout()
# plt.show()

### Setting independent and dependent variables and removing irrelevant columns

In [5]:
# X = dataset.drop(columns=['vote_average', 'budget', 'revenue'])
# y = dataset['vote_average']

# X['genres'] = X['genres'].str.split(' ')
# print(X['genres'])

# print(X['release_date'])

# X['release_date'] = pd.to_datetime(X['release_date'], format='%Y-%m-%d')
# X['release_date'] = X['release_date'].dt.year

# print(X['release_date'])


### Filling in missing data

In [6]:
# import numpy as np
# from sklearn.impute import SimpleImputer 
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# numeric_columns = [3, 8, 12, 17]
# # print(X[:, 17])

# X = np.array(X)

# for i in numeric_columns:
#     X[:, i] = imputer.fit_transform(X[:, i].reshape(-1, 1)).flatten()

# print(X)

# imputer.fit(X[:, numeric_columns])
# X[:, numeric_columns] = imputer.transform(X[:, numeric_columns])

### Encoding the data

#### Encoding the Independent Variable

In [7]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

# # Fill NaN values in 'genres' column with an empty list
# # df['genres'] = df['genres'].apply(lambda x: x if isinstance(x, list) else [])


# mlb = MultiLabelBinarizer()
# encoded_genres = mlb.fit_transform(df['genres'])

# encoded_genres_df = pd.DataFrame(encoded_genres, columns=mlb.classes_)


# # encoding_indices = [2, 4, 6, 9, 10, 15, 17, 18, 19]
# categorical_columns = [6, 10, 11, 15, 16]

# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), categorical_columns)], remainder='passthrough')

# X = ct.fit_transform(X)
# print(X)

# encoded_df = pd.DataFrame(X, columns=ct.get_feature_names_out())
# encoded_df = pd.concat([encoded_df, encoded_genres_df], axis=1)
# print(encoded_df)
# summary_statistics = encoded_df.describe(include='all')
# print(summary_statictics)


df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
5,5,258000000,Fantasy Action Adventure,http://www.sonypictures.com/movies/spider-man3/,559,dual identity amnesia sandstorm love of one's ...,en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,...,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576,Tobey Maguire Kirsten Dunst James Franco Thoma...,"[{'name': 'Francine Maisler', 'gender': 1, 'de...",Sam Raimi
6,6,260000000,Animation Family,http://disney.go.com/disneypictures/tangled/,38757,hostage magic horse fairy tale musical,en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,...,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They're taking adventure to new lengths.,Tangled,7.4,3330,Zachary Levi Mandy Moore Donna Murphy Ron Perl...,"[{'name': 'John Lasseter', 'gender': 2, 'depar...",Byron Howard
7,7,280000000,Action Adventure Science Fiction,http://marvel.com/movies/movie/193/avengers_ag...,99861,marvel comic sequel superhero based on comic b...,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,134.279229,...,141.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Age Has Come.,Avengers: Age of Ultron,7.3,6767,Robert Downey Jr. Chris Hemsworth Mark Ruffalo...,"[{'name': 'Danny Elfman', 'gender': 2, 'depart...",Joss Whedon
8,8,250000000,Adventure Fantasy Family,http://harrypotter.warnerbros.com/harrypottera...,767,witch magic broom

## Tokenising and normalising text data for vectoration

In [21]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


expected_columns = ['genres', 'keywords', 'cast', 'director']
df = df[[col for col in df.columns if col in expected_columns or col not in expected_columns]]



df['genres'] = df['genres'].apply(lambda x: x.lower().split(' ') if isinstance(x, str) else x)
df['keywords'] = df['keywords'].apply(lambda x: x.lower().split(' ') if isinstance(x, str) else x)


# Making a function for the cast which seperates two words into a list item

def split_into_pairs(text):
    if isinstance(text, str):
        # Use regular expression to find pairs of words
        return re.findall(r'\b\w+\s+\w+\b', text)
    return text

df['cast'] = df['cast'].apply(split_into_pairs)




# df




In [18]:
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-4o')


for row in df['genres']:
    if isinstance(row, list):
        for list_item in row:
            # print(list_item)
            # print(encoding.encode(list_item))
            df['encoded_genres'] = df['genres'].apply(lambda x: [encoding.encode(item) for item in x] if isinstance(x, list) else x)

print(df['genres'].iloc[6])
df['encoded_genres'].iloc[6]

['animation', 'family']


[[24205], [39832]]

### Encoding keywords column

In [26]:
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-4o')


for row in df['keywords']:
    if isinstance(row, list):
        for list_item in row:
            print(list_item)
            print(encoding.encode(list_item))
            df['encoded_keywords'] = df['keywords'].apply(lambda x: [encoding.encode(item) for item in x] if isinstance(x, list) else x)

# print(df['keywords'].iloc[6])
# df['encoded_keywords'].iloc[6]

# df

culture
[132158]
clash
[565, 1229]
future
[48758]
space
[8775]
war
[9126]
space
[8775]
colony
[1798, 4490]
society
[92442, 4343]
ocean
[78, 14838]
drug
[135295]
abuse
[378, 1821]
exotic
[490, 14682]
island
[276, 2117]
east
[157488]
india
[123520]
trading
[371, 2984]
company
[13003]
love
[55689]
of
[1440]
one's
[690, 885]
life
[19652]
spy
[57255]
based
[52911]
on
[263]
novel
[13802, 296]
secret
[37612]
agent
[16932]
sequel
[344, 15590]
mi6
[3900, 21]
dc
[13964]
comics
[639, 1541]
crime
[151152]
fighter
[147618]
terrorist
[143104, 421]
secret
[37612]
identity
[34010]
burglar
[11535, 3087]
based
[52911]
on
[263]
novel
[13802, 296]
mars
[130925]
medallion
[3217, 586, 294]
space
[8775]
travel
[94680]
princess
[144963, 1185]
dual
[159972]
identity
[34010]
amnesia
[313, 171273]
sandstorm
[163941, 40110]
love
[55689]
of
[1440]
one's
[690, 885]
life
[19652]
forgiveness
[1938, 70, 17140]
hostage
[6328, 477]
magic
[82292]
horse
[105889]
fairy
[53923, 88]
tale
[83, 1167]
musical
[40366, 933]
marve

[[6328, 477], [82292], [105889], [53923, 88], [83, 1167], [40366, 933]]